In [1]:
!pip install pymysql
!pip install pymssql

import LibData
import LibUtils
import LibML
import LibFile
import LibEE

import pandas as pd
import numpy as np


     |████████████████████████████████| 51kB 3.1MB/s 
     |████████████████████████████████| 1.3MB 6.3MB/s 


In [3]:

jobName = '201216_Train_split1'
# jobName = '201216_Train_pilot'

x, y = LibData.GetXYBase(jobName)
x = LibData.GetMoneyness(x)
x = LibData.GetIrReduction(x)
x = LibData.GetAttainedAge(x)
x, emb_prodCode = LibEE.GetXEmbApplied(3, x, y, ['ProdCode'])

print('Bef RF/XGBoost')
print(LibML.GetScoresCV(x,y,LibML.RandomForestDefault(len(x.columns)), 5), 
    LibML.GetScoresCV(x,y,LibML.XGBoostDefault(), 5))

Bef RF/XGBoost
start learning 1 2021-01-14 12:27... end learning 2021-01-14 12:50
start learning 2 2021-01-14 12:50... end learning 2021-01-14 13:12
start learning 3 2021-01-14 13:12... end learning 2021-01-14 13:35
start learning 4 2021-01-14 13:35... end learning 2021-01-14 13:57
start learning 5 2021-01-14 13:57... end learning 2021-01-14 14:20
start learning 1 2021-01-14 14:20... end learning 2021-01-14 14:23
start learning 2 2021-01-14 14:23... end learning 2021-01-14 14:26
start learning 3 2021-01-14 14:26... end learning 2021-01-14 14:29
start learning 4 2021-01-14 14:29... end learning 2021-01-14 14:33
start learning 5 2021-01-14 14:33... end learning 2021-01-14 14:36
[123.11, 0.1472, 0.9501] [115.18, 0.1377, 0.9696]


In [4]:

def Intersect(xFrom, cols, newColName, binsCnt, dim=3):
    x = xFrom.copy(True)
    
    cols_ = []
    for col in cols:
        x = LibEE.GetDigitized(x, col, binsCnt, replace=False)
        cols_.append(col+'_')

    x = LibEE.GetXIntersected(x, cols_, newColName, delOrgCols=True)
    x, emb_inter = LibEE.GetXEmbApplied(dim, x, y, [newColName])
    return x

def PrintRun(x, y):
    scaler, x = LibUtils.ScaleStandard(x)
    print('inter RF/XGBoost')
    print(LibML.GetScoresCV(x,y,LibML.RandomForestDefault(len(x.columns)), 5), 
          LibML.GetScoresCV(x,y,LibML.XGBoostDefault(), 5), '\n')

In [ ]:
binsCnt = 10

######### 상호작용항 1
x2 = Intersect(x, ['ElapsedMth', 'AttainedAge'], 'Inter_EM_AA', binsCnt=binsCnt)
PrintRun(x2, y)

######### 상호작용항 2
x2 = Intersect(x, ['ElapsedMth', 'StartAge'], 'Inter_EM_SA', binsCnt=binsCnt)
PrintRun(x2, y)

######### 상호작용항 3
x2 = Intersect(x, ['ElapsedMth', 'PremYr'], 'Inter_EM_SA', binsCnt=binsCnt)
PrintRun(x2, y)

######### 상호작용항 4
x2 = Intersect(x, ['AttainedAge', 'StartAge'], 'Inter_AA_SA', binsCnt=binsCnt)
PrintRun(x2, y)

######### 상호작용항 5
x2 = Intersect(x, ['AttainedAge', 'PremYr'], 'Inter_AA_PY', binsCnt=binsCnt)
PrintRun(x2, y)

######### 상호작용항 6
x2 = Intersect(x, ['StartAge', 'PremYr'], 'Inter_SA_PY', binsCnt=binsCnt)
PrintRun(x2, y)

In [5]:
############ 상호작용 : 머니니스 + 20년물금리 + 완납여부

def GetIsInPayment(xBase):
    x = xBase.copy(True)
    x['IsInPayment'] = x.apply(lambda row: 0 if row['ElapsedMth']//12 >= row['PremYr'] else 1, axis=1)
    return x

x2 = GetIsInPayment(x)

x2 = Intersect(x2, ['Moneyness', 'IR14', 'IsInPayment'], 'Inter3', binsCnt=5)
scaler, x2 = LibUtils.ScaleStandard(x2)
PrintRun(x2, y)


inter RF/XGBoost
start learning 1 2021-01-14 14:37... end learning 2021-01-14 15:00
start learning 2 2021-01-14 15:00... end learning 2021-01-14 15:23
start learning 3 2021-01-14 15:23... end learning 2021-01-14 15:45
start learning 4 2021-01-14 15:45... end learning 2021-01-14 16:08
start learning 5 2021-01-14 16:08... end learning 2021-01-14 16:31
start learning 1 2021-01-14 16:31... end learning 2021-01-14 16:35
start learning 2 2021-01-14 16:35... end learning 2021-01-14 16:38
start learning 3 2021-01-14 16:38... end learning 2021-01-14 16:42
start learning 4 2021-01-14 16:42... end learning 2021-01-14 16:46
start learning 5 2021-01-14 16:46... end learning 2021-01-14 16:49
[122.35, 0.1463, 0.9502] [114.06, 0.1364, 0.9709] 

